### 옵티마이저

* Optimizer 클래스: 매개변수 갱신을 위한 기반 클래스

In [1]:
import numpy as np
from dezero import Variable
import dezero.functions as F

np.random.seed(0)
x = np.random.rand(100, 1)
y = np.sin(2 * np.pi * x) + np.random.rand(100, 1)

In [ ]:
class Optimizer:    # 매개변수 갱신을 위한 기반 클래스
    def __init__(self):
        self.target = None
        self.hooks = []

    def setup(self, target):
        self.target = target
        return self

    def update(self):
        params = [p for p in self.target.params() if p.grad is not None]

        for f in self.hooks:
            f(params)

        for param in params:
            self.update_one(param)

    def update_one(self, param):        # 구체적인 매개변수 갱신
        raise NotImplementedError()     # 아담 등의 옵티마이저 여기에 구현

    def add_hook(self, f):
        self.hooks.append(f)

* SGD 클래스 구현

경사하강법으로 매개변수를 갱신하는 클래스를 구현

In [ ]:
class SGD(Optimizer):
    def __init__(self, lr=0.01):
        super().__init__()
        self.lr = lr

    def update_one(self, param):
        param.data -= self.lr * param.grad.data

* SGD 클래스를 사용하여 회귀 문제 풀기

In [ ]:
np.random.seed(0)
x = np.random.rand(100, 1)
y = np.sin(2 * np.pi * x) + np.random.rand(100, 1)

lr = 0.2
max_iter = 10000
hidden_size = 10

model = MLP((hidden_size, 1))
optimizer = optimizers.SGD(lr).setup(model)

for i in range(max_iter):
    y_pred = model(x)
    loss = F.mean_squared_error(y, y_pred)

    model.cleargrads()
    loss.backward()

    optimizer.update()
    if i % 1000 == 0:
        print(loss)

* 기울기를 이용한 최적화 기법<br>
: Momentum, AdaGrad, AdaDelta, Adam

1) W는 갱신할 가중치 매개변수 
2) 𝜕𝐿/𝜕𝑊은 기울기
3) η 는 학습률
4) v는 속도
5) αv 는 물체가 아무런 힘을 받지 않을때 서서히 감속시키는 역할

* MomentumSGD 구현

In [ ]:
class MomentumSGD(Optimizer):
    def __init__(self, lr=0.01, momentum=0.9):
        super().__init__()
        self.lr = lr
        self.momentum = momentum
        self.vs = {}

    def update_one(self, param):
        v_key = id(param)
        if v_key not in self.vs:
            xp = cuda.get_array_module(param.data)
            self.vs[v_key] = xp.zeros_like(param.data)

        v = self.vs[v_key]
        v *= self.momentum
        v -= self.lr * param.grad.data
        param.data += v

###  다중 클래스(Multi-class Classification) 분류

: 여러 클래스로 분류하는 문제

* 신경망으로 다중 클래스 분류

선형 회귀 때 이용한 신경망을 그대로 사용

In [ ]:
from dezero.models import MLP

model = MLP((10, 3))
x = Variable(np.array([[0.2, -0.4]]))
y = model(x)
p = softmax1d(y)

* 소프트맥스 함수

: 신경망 출력이 수치인데, 이 수치를 확률로 변환, 총합이 1

In [ ]:
def softmax1d(x):
    x = as_variable(x)
    y = F.exp(x)    
    sum_y = F.sum(y)
    return y / sum_y

* 배치(batch) 데이터에도 소프트맥수 함수 적용


In [ ]:
def softmax_simple(x, axis=1):
    x = as_variable(x)
    y = exp(x)
    sum_y = sum(y, axis=axis, keepdims=True)
    return y / sum_y

*교차 엔트로피 오차 구현<br>
: 정답에 해당하는 클래스면 1로, 그렇지 않으면 0으로 기록

In [ ]:
def softmax_cross_entropy_simple(x, t):
    x, t = as_variable(x), as_variable(t)
    N = x.shape[0]
    p = softmax(x)
    p = clip(p, 1e-15, 1.0)  # x_min 이하면 x_min 으로 변환하고, x_max 이상이면 x_max로 변환
    log_p = log(p)
    tlog_p = log_p[np.arange(N), t.data]
    y = -1 * sum(tlog_p) / N
    return y

### 다중 클래스 분류 수행

: 스파이럴 데이터셋을 사용하여 다중 클래스 분류 실제 수행

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import dezero
from dezero import optimizers
import dezero.functions as F
from dezero.models import MLP

# 하이퍼파라미터 설정
max_epoch = 300
batch_size = 30
hidden_size = 10
lr = 1.0
#모델과 옵티마이저를 생성
x, t = dezero.datasets.get_spiral(train=True)
model = MLP((hidden_size, 3))
optimizer = optimizers.SGD(lr).setup(model)


In [ ]:
data_size = len(x)
max_iter = math.ceil(data_size / batch_size)

for epoch in range(max_epoch):
    # 인덱스를 무작위로 섞음
    index = np.random.permutation(data_size)
    sum_loss = 0

    for i in range(max_iter): #  미니배치 생성
        batch_index = index[i * batch_size:(i + 1) * batch_size]
        batch_x = x[batch_index]
        batch_t = t[batch_index]
        # 기울기를 구하고 매개변수 갱신
        y = model(batch_x)
        loss = F.softmax_cross_entropy(y, batch_t)
        model.cleargrads()
        loss.backward()
        optimizer.update()

        sum_loss += float(loss.data) * len(batch_t)

    # 에포크마다 손실 함수 결과 출력
    avg_loss = sum_loss / data_size
    print('epoch %d, loss %.2f' % (epoch + 1, avg_loss))

### 대규모 데이터셋 처리

: 거대한 데이터를 하나의 ndarray 인스턴스로 처리하려면전용 클래스 생성

* Dataset 클래스 구현

In [ ]:
class Dataset: # 기반 클래스
    def __init__(self, train=True, transform=None, target_transform=None):
        self.train = train
        self.transform = transform
        self.target_transform = target_transform
        if self.transform is None:
            self.transform = lambda x: x
        if self.target_transform is None:
            self.target_transform = lambda x: x

        self.data = None
        self.label = None
        self.prepare()

    def __getitem__(self, index):
        assert np.isscalar(index)
        if self.label is None:
            return self.transform(self.data[index]), None
        else:
            return self.transform(self.data[index]),\
                   self.target_transform(self.label[index])

    def __len__(self):
        return len(self.data)

    def prepare(self):
        pass

# 어텐션(Attention)

### 기존의 seq2seq

* Encoder에서 마지막 timestep의 hidden state만을 Decoder의 입력으로 사용했다.
* 기본적인 모델의 경우, 인코더의 출력이 고정 길이 벡터<br>
-> 엄청난 길이의 문장이 입력되었을 때
필요한 정보가 벡터에 다 담기지 못하게 됩니다.


### 개선된 seq2seq

* 각 timestep 마다 hidden state의 행렬인 hs 전부를 활용할 수 있도록 Decoder를 개선한다.
* 앞서 개선된 인코더를 사용하여 얻은 히든 스테이트를 이용하여
디코더 과정에서 어떤 단어들끼리 서로 연관되어 있는지 그 대응관계를 모델에 학습시킨다.

-> 따라서, 필요한 정보에만 주목하여 그 정보로부터 시계열 변환을 수행하는 것이 Decoder의 목표이며, 이러한 구조를 Attention이라 한다

### 어텐션 구조
: 가중치 계산 계층, 선택작업계층, 결합 계층 총 3가지가 있다

가중치 계산층에서는 각 단어에 대해서 그것이 얼마나 중요한지를 나타내는 ‘가중치 a’를 구한다.

선택작업계층에서는 인코더가 내뱉은 히든 스테이트와 가중치 계산층에서 구한 가중치를 더한다 

* 가중치 계산층

In [ ]:
class WeightSum:
    def __init__(self):
        self.params, self.grads = [], []    #(1)
        self.cache = None       #(2)
        
    def forward(self, hs, a):   
        N, T, H = hs.shape  
        
        ar= a.reshape(N, T, 1)  #(4) 
        t = hs * ar             #(5)
        c = np.sum(t, axis=1)   #(6)
        
        self.cache = (hs, ar)   #(7)
        return c
    
    def backward(self, dc):
        hs, ar = self.cache                         # (8)
        N, T, H = hs.shape                          # (9)
        dt = dc.reshape(N, 1, H).repeat(T, axis=1)  # (10)
        dar = dt * hs                               # (11)
        dhs = dt * ar                               # (12)
        da = np.sum(dar, axis=2)                    # (13)
        
        return dhs, da

(1) 매개변수(params)와 미분값(grads)를 저장하는 리스트<br>
(2) 순전파(forward) 단계에서 사용된 값을 저장하여 역전파(backward) 단계에서 사용<br>
(3) 순전파. hs는 hidden state, N은 batch size, <br>T는 시퀀스 길이, H는 은닉층 수, a는 어텐션 가중치<br>
(4) 어텐션 가중치를 재배열하여 각 시퀀스 요소에 대응하도록 합<br>
(5) hidden states와 가중치를 곱하여 가중치를 적용<br>
(6) 차원에 대하여 합산하여 가중합을 계산<br>
(7) 역전파 단계에서 사용할 값을 저장

(8) 캐시 값 불러오기<br>
(9) 출력 형상 추출<br>
(10) 출력 미분값 재배열<br>
(11) 가중치에 대한 미분값 계산<br>
(12) 히든 상태에 대한 미분값 계산<br>
(13) 어텐션 가중치의 미분값 합산

* 선택작업계층

In [ ]:
class AttentionWeight:
    def __init__(self):
        self.params, self.grads = [], []    #(1)
        self.softmax = Softmax()            #(2)
        self.cache = None                   #(3)
        
    def forward(self, hs, h):
        N, T, H = hs.shape  

        hr = h.reshape(N, 1, H)
        t = hs * hr
        s = np.sum(t, axis=2)               #(4)
        a = self.softmax.forward(s)         #(5)

        self.cache = (hs, hr)
        return a

    def backward(self, da):
        hs, hr = self.cache                 #(6)
        N, T, H = hs.shape

        ds = self.softmax.backward(da)      #(7)
        dt = ds.reshape(N, T, 1).repeat(H, axis=2)
        dhs = dt * hr
        dhr = dt * hs
        dh = np.sum(dhr, axis=1)            #(8)

        return dhs, dh

(1) 불러올 값들 불러오기. 매개변수, 미분값, 가중치합 함수, 가중치 계산 함수<br>
(2) 계산된 어텐션 가중치를 저장<br>
(3) 어텐션 가중치 계산, 가중합 계산<br>
(4) 어텐션 가중치 저장<br>
(5) 가중합 반환 <br>
(6) dout를 가중합 함수에 대해 역전파하여 어텐션 가중치에 대한 미분값(da)와 dhs0을 계산<br>
(7) 미분값 da를 역전파하여 입력 시퀀스에 대한 미분값(dhs1)과 현재 차원에 대한 미분값(dh)을 계산<br>
(8) 두 단계에서 계산된 입력 시퀀스에 대한 미분값을 합하여 최종 입력 시퀀스에 대한 미분값 계산

- 가중합 역전파를 통해 어텐션 가중치와 입력 시퀀스에 대한 그라디언트를 계산한 후, 어텐션 가중치 역전파를 통해 입력 시퀀스와 현재 히든 상태에 대한 그라디언트를 계산

In [ ]:
class Attention:
    def __init__(self):
        self.params, self.grads = [], []                #(1)
        self.attention_weight_layer = AttentionWeight()
        self.weight_sum_layer = WeightSum()
        self.attention_weight = None                    

    def forward(self, hs, h):
        a = self.attention_weight_layer.forward(hs, h)  #(2)
        out = self.weight_sum_layer.forward(hs, a)
        self.attention_weight = a                       #(3)
        return out                                      

    def backward(self, dout):
        dhs0, da = self.weight_sum_layer.backward(dout) #(5)
        dhs1, dh = self.attention_weight_layer.backward(da) #(6)
        dhs = dhs0 + dhs1                               #(7)
        return dhs, dh

초기화 메서드<br>
(1) 파라미터와 미분값, 캐시를 초기화,<br>
가중치 계산 함수, 가중합 함수, 어텐션 가중치를 초기화<br>
순전파 메서드<br>
(2) 인코더의 은닉상태 hs와 출력된 은닉상태 h를 가지고 가중치 계산, <br>그후에 가중합 은닉상태와 가중치를 통해 가중합<br>
(3) 나온 값 가중치는 어텐션 웨이트로 저장하고 가중합을 반환<br>
역전파 메서드<br>
(4) 미분된 가중합을 가중합 역전파를 거쳐 가중치 미분값과 미분된 히든 스테이트을 계산<br>
(5)미분된 가중치를 가중치 계층의 역전파를 거쳐 히든스테이트의 미분값 dh와  히든스테이츠의 추가적인 미분값 계산<br>
(6) 구한 히든스테이트의 미분값 0,1을 더하여 최종적으로 계산된 히든스테이츠의 미분값 dhs과 현재 히든 스테이츠의 미분값 dh를 반환

결론적으로 이런 구현들로 통해 구한 값들을 통해 필요한 정보에만 주목하여 더욱 정확한 결과를 내놓을 수 있다.
